這份 Notebook 示範用 alternative.me 取得 Crypto Fear & Greed Index，並進行分析。



## 取得 openAI key 在Colab 使用權限

In [1]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [2]:
import requests
import json
from pprint import pp

In [3]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=4000):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

## 取得 Crypto Fear & Greed Index

In [4]:
def get_crypto_fear_and_greed_index():
    url = 'https://api.alternative.me/fng/'

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        index_value = None

        if 'data' in data and len(data['data']) > 0:
            index_value = data['data'][0]['value']
        else:
            print("Error: Could not retrieve Crypto Fear & Greed Index.")
    except requests.exceptions.RequestException as e:
        print("Error:", e)

    return index_value

In [5]:
message = [
    {"role": "system", "content": """
You are Crypto analyst, an AI who is skilled in giving suggestions based on the Crypto Fear & Greed Index.
作為一名加密貨幣投資者，你需要開發一個加密貨幣投資策略，包括技術和基本分析。請針對加密貨幣的特性，創建一組交易規則，包括進入點和退出點、停損訂單和風險管理技術。
Constraints:
* Do not answer if user ask other finacial products not cryptocurrency
* Do not chat or ask.
* Do not explain any price-related problems.

Please always respond in Chinese Mandarin Taiwanese and Taiwan terms.
When mixing Chinese and English, add a whitespace between Chinese and English characters.
    """
    },
    {"role": "user", "content": f"""The Crypto Fear & Greed Index today is {str(get_crypto_fear_and_greed_index())}. Base on this, give me some advice."""}
]

result = get_completion(message, model="gpt-3.5-turbo-1106", max_tokens=4096)
print(result)

根據目前的加密貨幣恐懼與貪婪指數為77，這代表市場情緒較為貪婪。在這種情況下，投資者可能要謹慎一些，可以考慮部分獲利了結，或者設定止損訂單以控制風險。同時也建議密切關注市場動向，隨時準備應對可能的變化。


## 取得現有加密貨幣清單

In [6]:
import requests

def get_crypto_list():
    url = "https://api.coingecko.com/api/v3/coins/list"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()  # Returns a list of dictionaries with 'id', 'symbol', 'name'
    except requests.RequestException as e:
        print(f"Error fetching cryptocurrency list: {e}")
        return None

crypto_list = get_crypto_list()

if crypto_list:
    for crypto in crypto_list[:10]:  # Print the first 10 cryptocurrencies to keep the output manageable
        print(f"ID: {crypto['id']}, Symbol: {crypto['symbol']}, Name: {crypto['name']}")
else:
    print("Failed to fetch the cryptocurrency list.")

ID: 01coin, Symbol: zoc, Name: 01coin
ID: 0chain, Symbol: zcn, Name: Zus
ID: 0-knowledge-network, Symbol: 0kn, Name: 0 Knowledge Network
ID: 0-mee, Symbol: ome, Name: O-MEE
ID: 0vix-protocol, Symbol: vix, Name: 0VIX Protocol
ID: 0vm, Symbol: zerovm, Name: 0VM
ID: 0x, Symbol: zrx, Name: 0x Protocol
ID: 0x0-ai-ai-smart-contract, Symbol: 0x0, Name: 0x0.ai: AI Smart Contract
ID: 0x1-tools-ai-multi-tool, Symbol: 0x1, Name: 0x1.tools: AI Multi-tool
ID: 0x404, Symbol: xfour, Name: 0x404


In [7]:
# bcs openai set rate limit, so filter first
def filter_cryptos_by_first_and_second_letter(crypto_list, first_letter, second_letter):
    return [crypto['id'] for crypto in crypto_list if crypto['id'][0] == first_letter and len(crypto['id']) > 1 and crypto['id'][1] == second_letter]

## Prompt: Matching coins from user input

In [8]:
def map_coin_prompt(user_input):
  map_coin_message = [
    {"role": "system", "content": f"""
Help to map the user input to the English coin_name.
No need to return Chinese name.
For examples:
1. User_input: "我應該投資比特幣嗎？" Response: Bitcoin
2. User_input: "以太幣現在的市值如何？" Response: Ethereum
    """
    },
    {"role": "user", "content": user_input}
]

  coin_name_result = get_completion(map_coin_message, model="gpt-3.5-turbo-1106", max_tokens=4096)
  return coin_name_result

test_case = map_coin_prompt("該支持馬斯克的狗狗幣嗎？")
print(test_case)
filtered_list = filter_cryptos_by_first_and_second_letter(crypto_list, test_case[0].lower(), test_case[1].lower())
print(filtered_list)

Dogecoin
['dobi', 'dock', 'doctor-evil', 'docuchain', 'documentchain', 'dodo', 'dodo-2', 'doeg-wif-rerart', 'dog-3', 'dogai', 'dogami', 'dogcoin', 'dog-collar', 'dog-coq', 'doge-1', 'doge-1-2', 'doge-1-mission-to-the-moon', 'doge-1-moon-mission', 'doge-1satellite', 'doge-2-0', 'doge2-0', 'doge69', 'dogeai', 'dogebonk', 'dogebonk-eth', 'dogebonk-on-sol', 'dogeboy', 'dogecash', 'doge-ceo', 'dogeceomeme', 'dogechain', 'dogeclub', 'dogecoin', 'dogecoin-2', 'dogecola', 'dogecube', 'dogedi', 'dogedragon', 'doge-eat-doge', 'doge-floki-2-0', 'doge-floki-coin', 'doge-floki-coin-2', 'dogefood', 'dogegayson', 'dogegf', 'doge-grok', 'dogegrow', 'doge-inu', 'doge-kaki', 'dogeking', 'dogelana', 'doge-legion', 'dogelogy', 'dogelon-classic', 'dogelon-mars', 'dogelon-mars-2-0', 'dogelon-mars-wormhole', 'doge-lumens', 'dogemeta', 'dogemon-go', 'dogemoon', 'dogemoon-2', 'doge-of-grok-ai', 'dogeon', 'doge-on-pulsechain', 'dogepad-finance', 'doge-protocol', 'dogeshrek', 'dogeswap', 'dogether', 'doge-token'

In [15]:
def map_coin_to_id_prompt(filtered_input):
  map_coin_to_id_message = [
    {"role": "system", "content": f"""
Help to find the most relevant coin_id.
No need to return Chinese name.
Only return single id, no need to answer in Sentence.
For examples:
1. User_input: "Bitcoin" Response: bitcoin
    """
    },
    {"role": "user", "content": filtered_input}
]

  coin_id_result = get_completion(map_coin_to_id_message, model="gpt-3.5-turbo-1106", max_tokens=4096)
  return coin_id_result

test_id_case = map_coin_to_id_prompt("Dogecoin")
print(test_id_case)

test_id_case1 = map_coin_to_id_prompt("Solana")
print(test_id_case1)

dogecoin
solana


## 取得現貨價格

In [10]:
def get_crypto_price(coin_name):
    coin_id = coin_name.lower()
    url = f'https://api.coingecko.com/api/v3/simple/price?ids={coin_id}&vs_currencies=usd'

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        if coin_id in data:
            return data[coin_id]['usd']
        else:
            print(f"Error: Could not retrieve price data for {coin_name.capitalize()}")
            return None
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return None

print(get_crypto_price(test_id_case))

0.150151


## Chatbot UI

In [11]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.9 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=

In [ ]:
def chat(input_message, chat_history=[]):
  if not chat_history:
        # If empty, this is the first user interaction, add an instructional message
        chat_history.append(("", "你好！我是AI助手，可以提供加密貨幣的價格, 並根據貪婪指數分析。請問你想要查詢哪種加密貨幣的當前價格？"))

  openai_messages = [
  {"role": "system", "content": "你是一個AI助手, 可以提供加密貨幣的價格, 並根據貪婪指數分析" }
]

  # 把 gradio 紀錄的 chat_history 轉成 OpenAI 的 messages 格式
  for human, assistant in chat_history:
      openai_messages.append({"role": "user", "content": human })
      openai_messages.append({"role": "assistant", "content": assistant})

  openai_messages.append({"role": "user", "content": input_message})


  fear_index_analysis = result

  coin_full_name = map_coin_prompt(input_message)
  # filtered_list = filter_cryptos_by_first_and_second_letter(crypto_list, coin_full_name[0].lower(), coin_full_name[1].lower())
  coin_id = map_coin_to_id_prompt(coin_full_name)

  price_content = str(get_crypto_price(coin_id))
  return f"目前該幣價格為 {price_content} USD。 " + fear_index_analysis

import gradio as gr
gr.close_all()

demo2 = gr.ChatInterface(fn=chat)
demo2.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://5efc006d6b9a693757.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
